In [1]:
def preprocess_string(s):
    s = s.lower()
    plaintext = ''
    for i in s:
        temp = ord(i)
        if (temp < 123 and temp > 96 or temp < 91 and temp > 64
            or temp < 58 and temp > 47):
            plaintext += i
    return plaintext

In [2]:
def char_to_bits(s, char_size = 4):
    ans = bin(int(s, 16))[2:]
    ans = '0' * (char_size - len(ans)) + ans 
    return ans

In [3]:
def char_chunk_to_bits(s, char_size = 4):
    ans = ''
    for i in s:
        ans += char_to_bits(i, char_size)
    return ans

In [4]:
def create_chunks(s, chunk_size, char_size = 4, bogus = 'F'):
    char_count = chunk_size // char_size
    increment = len(s) % char_count
    if increment:
        s += ((len(s) // char_count + 1) * char_count - len(s)) * bogus
    chunks = [s[i:i + char_count] for i in range(0, len(s), char_count)]
    return chunks

In [5]:
def get_bit_chunks(s, chunk_size, char_size = 4, bogus = 'F'):
    s = preprocess_string(s)
    chunks = create_chunks(s, chunk_size, char_size, bogus)
    bit_chunks = [char_chunk_to_bits(i, char_size) for i in chunks]
    return bit_chunks

## 1. DES

In [7]:
def shift_left(s, n):
    for i in range(n):
        s = s[1:] + s[0]
    return s

def permute(s, key):
    ans = []
    for i in key:
        ans.append(s[i - 1])
    return ''.join(ans)

def create_keys(key_with_parity_bits):
    parity_bit_drop_table = [57, 49, 41, 33, 25, 17, 9,
                         1, 58, 50, 42, 34, 26, 18,
                         10, 2, 59, 51, 43, 35, 27,
                         19, 11, 3, 60, 52, 44, 36,
                         63, 55, 47, 39, 31, 23, 15,
                         7, 62, 54, 46, 38, 30, 22,
                         14, 6, 61, 53, 45, 37, 29,
                         21, 13, 5, 28, 20, 12, 4]
    
    number_of_bits_shifts = [1, 1, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 1]
    
    key_compression_table = [14, 17, 11, 24, 1, 5, 3, 28,
                        15, 6, 21, 10, 23, 19, 12, 4,
                        26, 8, 16, 7, 27, 20, 13, 2,
                        41, 52, 31, 37, 47, 55, 30, 40,
                        51, 45, 33, 48, 44, 49, 39, 56,
                        34, 53, 46, 42, 50, 36, 29, 32]
    
    cipher_key = permute(key_with_parity_bits, parity_bit_drop_table)
    
    left_key = cipher_key[0:28]
    right_key = cipher_key[28:56]
    
    keys = []
    
    for i in range(16):
        left_key = shift_left(left_key, number_of_bits_shifts[i])
        right_key = shift_left(right_key, number_of_bits_shifts[i])
        keys.append(permute(left_key + right_key, key_compression_table))
        
    return keys

In [8]:
def permute(s, key):
    ans = []
    for i in key:
        ans.append(s[i - 1])
    return ''.join(ans)

def char_to_bits(s, char_size = 4):
    ans = bin(int(str(s), 16))[2:]
    ans = '0' * (char_size - len(ans)) + ans 
    return ans

def substitute(s, key):
    answer = ''
    for i in range(0, 48, 6):
        vertical_entry = s[i] + s[i + 5]
        horizontal_entry = s[i + 1:i + 5]
        temp = key[i // 6][int(vertical_entry, 2)][int(horizontal_entry, 2)]
        temp = bin(temp)[2:]
        temp = '0' * (4 - len(temp)) + temp
        answer += temp
    return answer
    
def round_normal_without_swapping(P, key):
    
    left_plaintext = P[0:32]
    right_plaintext = P[32:64]
    
    right_expansion_p_box_table = [32, 1, 2, 3, 4, 5,
                                 4, 5, 6, 7, 8, 9,
                                 8, 9, 10, 11, 12, 13,
                                 12, 13, 14, 15, 16, 17,
                                 16, 17, 18, 19, 20, 21,
                                 20, 21, 22, 23, 24, 25,
                                 24, 25, 26, 27, 28, 29,
                                 28, 29, 30, 31, 32, 1]

    
    right_expanded_plaintext = permute(right_plaintext, right_expansion_p_box_table)
    
    xored_right_text = ''
    for i, j in zip(right_expanded_plaintext, key):
        xored_right_text += str(int(i) ^ int(j))
        
    substitution_table = [
        [[14, 4, 13, 1, 2, 15, 11, 8, 3, 10, 6, 12, 5, 9, 0, 7],
         [0, 15, 7, 4, 14, 2, 13, 1, 10, 6, 12, 11, 9, 5, 3, 8],
         [4, 1, 14, 8, 13, 6, 2, 11, 15, 12, 9, 7, 3, 10, 5, 0],
         [15, 12, 8, 2, 4, 9, 1, 7, 5, 11, 3, 14, 10, 0, 6, 13],
        ],
        [[15, 1, 8, 14, 6, 11, 3, 4, 9, 7, 2, 13, 12, 0, 5, 10],
         [3, 13, 4, 7, 15, 2, 8, 14, 12, 0, 1, 10, 6, 9, 11, 5],
         [0, 14, 7, 11, 10, 4, 13, 1, 5, 8, 12, 6, 9, 3, 2, 15],
         [13, 8, 10, 1, 3, 15, 4, 2, 11, 6, 7, 12, 0, 5, 14, 9],
        ],
        [[10, 0, 9, 14, 6, 3, 15, 5, 1, 13, 12, 7, 11, 4, 2, 8],
         [13, 7, 0, 9, 3, 4, 6, 10, 2, 8, 5, 14, 12, 11, 15, 1],
         [13, 6, 4, 9, 8, 15, 3, 0, 11, 1, 2, 12, 5, 10, 14, 7],
         [1, 10, 13, 0, 6, 9, 8, 7, 4, 15, 14, 3, 11, 5, 2, 12],
        ],
        [[7, 13, 14, 3, 0, 6, 9, 10, 1, 2, 8, 5, 11, 12, 4, 15],
         [13, 8, 11, 5, 6, 15, 0, 3, 4, 7, 2, 12, 1, 10, 14, 9],
         [10, 6, 9, 0, 12, 11, 7, 13, 15, 1, 3, 14, 5, 2, 8, 4],
         [3, 15, 0, 6, 10, 1, 13, 8, 9, 4, 5, 11, 12, 7, 2, 14],
        ],  
        [[2, 12, 4, 1, 7, 10, 11, 6, 8, 5, 3, 15, 13, 0, 14, 9],
         [14, 11, 2, 12, 4, 7, 13, 1, 5, 0, 15, 10, 3, 9, 8, 6],
         [4, 2, 1, 11, 10, 13, 7, 8, 15, 9, 12, 5, 6, 3, 0, 14],
         [11, 8, 12, 7, 1, 14, 2, 13, 6, 15, 0, 9, 10, 4, 5, 3],
        ], 
        [[12, 1, 10, 15, 9, 2, 6, 8, 0, 13, 3, 4, 14, 7, 5, 11],
         [10, 15, 4, 2, 7, 12, 9, 5, 6, 1, 13, 14, 0, 11, 3, 8],
         [9, 14, 15, 5, 2, 8, 12, 3, 7, 0, 4, 10, 1, 13, 11, 6],
         [4, 3, 2, 12, 9, 5, 15, 10, 11, 14, 1, 7, 6, 0, 8, 13],
        ], 
        [[4, 11, 2, 14, 15, 0, 8, 13, 3, 12, 9, 7, 5, 10, 6, 1],
         [13, 0, 11, 7, 4, 9, 1, 10, 14, 3, 5, 12, 2, 15, 8, 6],
         [1, 4, 11, 13, 12, 3, 7, 14, 10, 15, 6, 8, 0, 5, 9, 2],
         [6, 11, 13, 8, 1, 4, 10, 7, 9, 5, 0, 15, 14, 2, 3, 12],
        ],
        [[13, 2, 8, 4, 6, 15, 11, 1, 10, 9, 3, 14, 5, 0, 12, 7],
         [1, 15, 13, 8, 10, 3, 7, 4, 12, 5, 6, 11, 0, 14, 9, 2],
         [7, 11, 4, 1, 9, 12, 14, 2, 0, 6, 10, 13, 15, 3, 5, 8],
         [2, 1, 14, 7, 4, 10, 8, 13, 15, 12, 9, 0, 3, 5, 6, 11],
        ]
        ]
    
    straight_permutation_table = [16, 7, 20, 21, 29, 12, 28, 17,
                                 1, 15, 23, 26, 5, 18, 31, 10,
                                 2, 8, 24, 14, 32, 27, 3, 9,
                                 19, 13, 30, 6, 22, 11, 4, 25]
    
    substituted_right_text = substitute(xored_right_text, substitution_table)
    des_function_out = permute(substituted_right_text, straight_permutation_table)
    xored_left_text = ''
    
    for i, j in zip(left_plaintext, des_function_out):
        xored_left_text += str(int(i) ^ int(j))
        

    return xored_left_text + right_plaintext
    
def round_normal(P, key):
    ciphertext = round_normal_without_swapping(P, key)
    return ciphertext[32:] + ciphertext[0:32]

def round_last(P, key):
    return round_normal_without_swapping(P, key)

In [9]:
def DES(plaintext, K):
    
    keys = create_keys(K)
    
    initial_permutation_table = [58, 50, 42, 34, 26, 18, 10, 2,
                                  60, 52, 44, 36, 28, 20, 12, 4,
                                  62, 54, 46, 38, 30, 22, 14, 6,
                                  64, 56, 48, 40, 32, 24, 16, 8,
                                  57, 49, 41, 33, 25, 17, 9, 1,
                                  59, 51, 43, 35, 27, 19, 11, 3,
                                  61, 53, 45, 37, 29, 21, 13, 5,
                                  63, 55, 47, 39, 31, 23, 15, 7]
    
    final_permutation_table = [40, 8, 48, 16, 56, 24, 64, 32,
                                39, 7, 47, 15, 55, 23, 63, 31,
                                38, 6, 46, 14, 54, 22, 62, 30,
                                37, 5, 45, 13, 53, 21, 61, 29,
                                36, 4, 44, 12, 52, 20, 60, 28,
                                35, 3, 43, 11, 51, 19, 59, 27,
                                34, 2, 42, 10, 50, 18, 58, 26,
                                33, 1, 41, 9, 49, 17, 57, 25]
    
    blocks = get_bit_chunks(P, 64)
    
    encrypted_blocks = []
    for i in blocks:
        permuted_text = permute(i, initial_permutation_table)
        cipher_text = permuted_text
        for i in range(15):
            cipher_text = round_normal(cipher_text, keys[i])
        cipher_text = round_last(cipher_text, keys[15])
        
        cipher_text_final = permute(cipher_text, final_permutation_table)
        encrypted_blocks.append(cipher_text_final)
    
    return ''.join(encrypted_blocks)

In [10]:
# P = input("Enter Plaintext: ")
# K = input("Enter DES Key: ")

P = "0123456789ABCDEF"
K = "0001001100110100010101110111100110011011101111001101111111110001"

ciphertext = DES(P, K)
print(ciphertext)

1000010111101000000100110101010000001111000010101011010000000101
